3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리해보기로 하였다.

In [33]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

In [34]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [35]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')

In [36]:
train_text[:100]

'id\tdocument\tlabel\n9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0\n3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1\n10265843'

In [37]:
a= train_text.split('\n')[1:][0]

a.split('\t')[2]

'0'

In [38]:
# X 값 추출 = 리뷰문장 == 피쳐
x_train = [ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0]
x_train[:3]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

In [39]:
#y값 추출  == target
y_train= np.array([ [int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)
y_train

(150000, 1)


array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [40]:
from konlpy.tag import Okt  #트위터에서 만든 한국어분석 메서드
from konlpy.tag import Kkma  #서울대에서 만든거 morphs== 형태소 분석


x_tokenized = [ ]
for i in x_train[:100]:
    x_tokenized.append( Okt().morphs( i ) ) 
    
x_tokenized[:5]

[['아', '더빙', '..', '진짜', '짜증나네요', '목소리'],
 ['흠',
  '...',
  '포스터',
  '보고',
  '초딩',
  '영화',
  '줄',
  '....',
  '오버',
  '연기',
  '조차',
  '가볍지',
  '않구나'],
 ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'],
 ['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정'],
 ['사이',
  '몬페',
  '그',
  '의',
  '익살스런',
  '연기',
  '가',
  '돋보였던',
  '영화',
  '!',
  '스파이더맨',
  '에서',
  '늙어',
  '보이기만',
  '했던',
  '커스틴',
  '던스트',
  '가',
  '너무나도',
  '이뻐',
  '보였다']]

. ! ? ;;; 은 는 이 가 => 불용어 제거
불용어 빈도수가 늘어남 = 중요도 판별하는 순위가 되기 때문에
=> TF-IDF  TF= 빈도수  IDF = 역 빈도수 

0 부정문들만 모여있는 문서
1 긍정문들만 모여있는 문서

=> TF-IDF 알고리즘은 두 문서에 겹치는 단어들 동시에 많이 쓰이는 단어들은 점수를 낮게
=> 겹치지않는 단어들은 점수를 높게 매겨준다.

In [41]:
import pandas as pd

x_train[:100] #1차원


y_train[:100]  #2차원 

x_df = pd.DataFrame(x_train[:1000])
# print(x_df)

y_df = pd.DataFrame(y_train[:1000])
# print(y_df)

# pd.DataFrame(x_df, y_df, columns=['x','y'], axis= 1)

df = pd.concat([x_df, y_df], axis=1)

# df
# df.rename(columns={0, 0 : 'x','y'})
# df_1= df[ (df.iloc[1:] ==0) ] 


bad = df[df.iloc[:,1]==0]
good = df[df.iloc[:,1]==1]

bad
# TF-IDF 단어 중요도 점수를 매긴다.

,0,0
0,아 더빙.. 진짜 짜증나네요 목소리,0
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
5,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,원작의 긴장감을 제대로 살려내지못했다.,0
...,...,...
993,마이클베이 최근작을 볼때마다 느낀다. 블록버스터 액션장르라도 탄탄한 시나리오는 영화...,0
994,0점은없나?1점은 너무 후하네,0
995,미달이는.. 연예계를 바라보는 태도를 고쳐야 한다.,0
996,파괴된 관객들.,0


In [42]:
# bad[:3]
good[:3]

,0,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
8,액션이 없는데도 재미 있는 몇안되는 영화,1


In [43]:
bad_text = ''
for i in bad.iloc[:,0]:
    bad_text += i
    
# print(bad_text)

good_text = ''
for i in good.iloc[:,0]:
    good_text += i
    
# print(good_text)

texts =[ bad_text , good_text ]
texts


['아 더빙.. 진짜 짜증나네요 목소리너무재밓었다그래서보는것을추천한다교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.원작의 긴장감을 제대로 살려내지못했다.별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐재미없다 지루하고. 같은 음식 영화인데도 바베트의 만찬하고 넘 차이남....바베트의 만찬은 이야기도 있고 음식 보는재미도 있는데 ; 이건 볼게없다 음식도 별로 안나오고, 핀란드 풍경이라도 구경할랫는데 그것도 별로 안나옴 ㅡㅡ주제는 좋은데 중반부터 지루하다다 짤랐을꺼야. 그래서 납득할 수 없었던거야.. 그럴꺼야.. 꼭 그랬던걸꺼야..카밀라벨 발연기졸쓰레기 진부하고말도안됌ㅋㅋ 아..시간아까워1%라도 기대했던 내가 죄인입니다 죄인입니다....키이라 나이틀리가 연기하고자 했던건 대체 정신장애일까 틱장애일까포스터는 있어보이는데 관객은 114명이네\'다 알바생인가 내용도 없고 무서운거도 없고 웃긴거도 하나도 없음 완전 별싱거운 영화.ㅇ.ㅇ내ㅇ시간 넘 아까움 .. . 완전 낚임평점에속지마시길시간낭비 돈낭비임리얼리티가 뛰어나긴 한데 큰 공감은 안간다. 이민기캐릭터는 정신의학상 분노조절장애 초기 증상일거다. 툭하면 사람패고 욕하고 물건 파손하고.. 조금 오바였음. 극 초반엔 신선했는데 가면 갈수록 이민기 정신상태 공감불가.마이너스는 왜없냐 ㅋ 뮤비 보고 영화수준 딱 알만하더군 ㅉㅉ 북한에서 이런거 만들라고 돈 대주던?난 우리영화를 사랑합니다....^^;야 세르게이! 작은고추의 매운맛을 보여주마! 포퐁저그 콩진호가 간다난또 저 꼬마애가 무슨 원한이 깊길래.,. 했더니 OO

In [44]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

vectorizer = CountVectorizer(tokenizer= Kkma().morphs )

# 요쯤에 불용어 제거 들어가야할것같은 느낌 => 생략
    
X = vectorizer.fit_transform( texts )

word_list = vectorizer.get_feature_names_out()
# word_list

# 오탈자, 잘 쓰지 않는말 같은경우는 가중치가 높아질수있다.
print(X.toarray())  #TF본다. 각문서에서의 빈도수를 확인한다.

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[[11  6  4 ...  1  0  3]
 [48 22  8 ...  1  1  4]]


In [45]:
idf = TfidfTransformer()
X_tfidf = idf.fit_transform( X )

print( X_tfidf.toarray( ) )

[[0.01199996 0.00654543 0.00436362 ... 0.00109091 0.         0.00327272]
 [0.05326131 0.02441143 0.00887688 ... 0.00110961 0.00155952 0.00443844]]


In [46]:
len(X_tfidf.toarray()[0])
len(X_tfidf.toarray()[1])

3468

In [47]:
# 딥러닝 모델
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
# from tensorflow.keras.

#딥러닝기반 언어(텍스트정제 => 숫자로된 시계열데이터로 들어온다 )처리모델 

model = Sequential()

#들어온 시퀀스데이터를 벡터로 변환한다.5000==단어사전크기, 600 차원은 클수록 좋다. 인풋랭스는 시퀀스데이터의 길이이다.
model.add( Embedding( 30, 200, input_length = 3468)) #입력층

model.add( LSTM( units = 50) )   #은닉층,  units: 뉴런갯수 많을수록 좋다.

#0부정,1긍정
model.add( Dense( 2, activation='softmax' ))  #출력층
#activation='softmax'이면 자동으로 loss='sparse_categorical_crossentropy'이다.


# compile: ex) c언어 소스코드 => 컴터가 못알아들음 => 그럼면 알아들을수있게 컴파일해준다.
#컴파일  , optimazer, loss ,metrics
# sparse_categorical_crossentropy는 categorical_crossentropy와 동일하지만, 정수 형태의 레이블을 사용
model.compile(optimizer='adam',  loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [48]:
y_train=np.array([
    [0],
    [1]
])

X_tfidf.toarray()

array([[0.01199996, 0.00654543, 0.00436362, ..., 0.00109091, 0.        ,
        0.00327272],
       [0.05326131, 0.02441143, 0.00887688, ..., 0.00110961, 0.00155952,
        0.00443844]])

In [49]:
model.fit( X_tfidf.toarray(), y_train, epochs=5, batch_size = 500 , validation_split = 0.2 )

Epoch 1/5
1/1 [==============================] - 9s 9s/step - loss: 0.6985 - acc: 0.0000e+00 - val_loss: 0.7525 - val_acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 3s 3s/step - loss: 0.6371 - acc: 1.0000 - val_loss: 0.8222 - val_acc: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 3s 3s/step - loss: 0.5789 - acc: 1.0000 - val_loss: 0.8994 - val_acc: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 3s 3s/step - loss: 0.5222 - acc: 1.0000 - val_loss: 0.9888 - val_acc: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 0.4653 - acc: 1.0000 - val_loss: 1.0960 - val_acc: 0.0000e+00


In [50]:
#테스트 문자열 생성
# test = ['프로젝트 개망했다.'] 
test = ['상당히 볼만했다. 재미있었다.']

test = vectorizer.transform(test)
test.toarray()

test = idf.transform( test )
test.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [51]:
model.predict(test.toarray())

1/1 [==============================] - 1s 1s/step


array([[0.6658059 , 0.33419412]], dtype=float32)